# 3: Assessing model performance and applicability

Author: Daniel Lusk

## Imports

In [ ]:
import pandas as pd
from utils.geostats import aoa
from utils.models import Model

%load_ext autoreload
%autoreload 2

## Area of Applicability (AOA)

## Model performance

Get the training run data from the results dataframe

In [ ]:
# Load a model
results_df = pd.read_csv("./results/training_results.csv").copy()
training_id = "2023-07-12_17-39-29"
results = results_df.loc[(results_df["Run ID"] == training_id)].sort_values(by="Test r-squared", ascending=False)

Load the best performing model

In [ ]:
row = results.iloc[0]
model = Model.from_results_row(row)

In [ ]:
pred = model.model.predict(model.X_test)
obs = model.y_test

import matplotlib.pyplot as plt


# plot the observed vs. predicted values using seaborn
sns.set_theme()
sns.set_style("whitegrid")
sns.set_context("paper")


fig, ax = plt.subplots()
p1 = min(min(pred), min(obs))
p2 = max(max(pred), max(obs))
ax.plot([p1, p2], [p1, p2], color="black", lw=0.5)
ax.scatter(pred, obs)


In [ ]:
from utils.datasets import GBIFBand

[b.readable for b in GBIFBand]